# 임직원 데이터 기반 퇴사 확률 예측

- 나이 그룹 (`age_group`): 직원의 나이를 30대 초반 이하(0)와 30대 중반 이상(1)으로 구분.
- 자녀 수 (`num_children`): 직원이 보유한 자녀 수 (나이에 따라 확률적으로 0~3명).
- 근무지 (`unfavorable_location`): 직원의 근무지가 기피 지역인지 여부 (1: 거리 10km 이상, 0: 거리 10km 이하).
- 직렬 (`is_support`): 직무가 지원 부서인지 여부 (1: 지원 부서, 0: 사업 부서).
- 전문 자격증 보유 여부 (`has_certification`): 직원이 전문 자격증을 보유했는지 여부 (1: 있음, 0: 없음).
- 야근 횟수 (`overtime_hours`): 지난 1년간 야근 횟수 (0~15회, OverTime 변수 기반).
- 고충 이용 여부 (`used_complaint_service`): 직원이 고충 처리 서비스를 이용했는지 여부 (1: 있음, 0: 없음).
- 퇴사 여부 (`attrition`): 직원의 퇴사 여부 (True: 퇴사, False: 재직).

데이터 원본: https://www.kaggle.com/datasets/pavansubhasht/ibm-hr-analytics-attrition-dataset/data

In [5]:
# 1. 패키지 불러오기 및 데이터 로드
import pandas as pd
import sweetviz as sv
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, precision_score, recall_score, accuracy_score, classification_report
import matplotlib.pyplot as plt
import seaborn as sns

# 데이터 로드
data = pd.read_excel('1조_데이터.xlsx')

In [ ]:
# 2. 데이터 구조 보기
print("데이터 구조:")
print(data.info())
print("\n데이터 샘플:")
print(data.head())

In [ ]:
# 3. Sweetviz를 활용한 EDA
eda_report = sv.analyze(data)
eda_report.show_notebook()

In [7]:
# 4. 데이터 분리
X = data.drop(columns=['attrition'])  # 독립 변수
y = data['attrition']                 # 종속 변수
# 수치형 변수만 선택 (Random Forest는 숫자 데이터를 요구)
X = pd.get_dummies(X, columns=['is_support'], drop_first=True)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [8]:
# 5. Random Forest 모델링
model = RandomForestClassifier(random_state=42, n_estimators=100)
model.fit(X_train, y_train)

# 예측
y_pred = model.predict(X_test)

In [ ]:
# 6. 결과 평가
conf_matrix = confusion_matrix(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)

print("Confusion Matrix:")
print(conf_matrix)
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

print(f"Precision: {precision:.2f}, Recall: {recall:.2f}, Accuracy: {accuracy:.2f}")

In [ ]:
# 7. Feature Importance 시각화
importances = model.feature_importances_
features = X.columns

plt.figure(figsize=(10, 6))
sns.barplot(x=importances, y=features)
plt.title('Feature Importance')
plt.xlabel('Importance Score')
plt.ylabel('Feature')
plt.show()